# Image Completion

## Load the CIFAR-10 dataset

In [12]:
from keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

## Generator model

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Reshape, UpSampling2D, Conv2D, LeakyReLU

def make_generator_model():
    model = Sequential()
    model.add(Dense(256, use_bias = False, input_shape=(100,)))
    model.add(LeakyReLU())
    
    model.add(Dense(8*8*256, use_bias = False))
    model.add(LeakyReLU())
    
    model.add(Reshape((8, 8, 256)))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256, (5, 5), strides =(1, 1), padding = 'same', use_bias = False))
    model.add(LeakyReLU())
    
    model.add(UpSampling2D())
    model.add(Conv2D(3, (5, 5), strides=(1, 1), padding = 'same', use_bias = False))
    
    return model
    

## Discriminator model

In [28]:
from keras.layers import Flatten, Dropout

def make_discriminator_model():
    model = Sequential()
    model.add(Conv2D(64, (5,5), strides=(2,2), padding = 'same', input_shape = [32, 32, 3]))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    
    model.add(Conv2D(128, (5, 5), strides=(2,2), padding = 'same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    model.add(Dense(1))
    
    return model

## Loss and Optimizers

In [29]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

## Training loop

In [41]:
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, 100])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training = True)
        
        real_output = discriminator(images, training = True)
        fake_output = discriminator(generated_images, training = True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

## Training process

In [42]:
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)

## Preprocessing and training

In [ ]:
train_images = (train_images - 127.5) / 127.5

BUFFER_SIZE = 60000
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

generator = make_generator_model()
discriminator = make_discriminator_model()

EPOCHS = 50
train(train_dataset, EPOCHS)